In [8]:
import sys
import os
sys.path.append("D:\Data\Legal-Retrieval\lexiops-copilot")
import asyncio
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from agent.state import AgentState
from langchain_openai import ChatOpenAI
from agent.nodes.planner_node import planner_node
from agent.nodes.synthesizer_node import synthesizer_node
from langgraph.prebuilt import create_react_agent
from langgraph.graph import StateGraph, MessagesState, START
from langgraph.prebuilt import ToolNode, tools_condition
import logging

logger = logging.getLogger(__name__)

# --- Tool nội bộ ---
from langchain.tools import tool
from datetime import datetime

@tool
def get_current_time() -> str:
    """Sử dụng để lấy thời gian hiện tại của hệ thống."""
    return datetime.now().isoformat()

# --- MCP client ---
from langchain_mcp_adapters.client import MultiServerMCPClient

<>:3: SyntaxWarning: invalid escape sequence '\D'
<>:3: SyntaxWarning: invalid escape sequence '\D'
C:\Users\Admin\AppData\Local\Temp\ipykernel_13236\3430067079.py:3: SyntaxWarning: invalid escape sequence '\D'
  sys.path.append("D:\Data\Legal-Retrieval\lexiops-copilot")


In [9]:
model =   ChatOpenAI(
        model="gpt-5-nano",  
        openai_api_key=os.getenv("OPENAI_API_KEY")
    )

In [10]:
import asyncio
from mcp.client.stdio import stdio_client
from mcp import ClientSession, StdioServerParameters
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.graph import StateGraph, MessagesState, START
from langgraph.prebuilt import ToolNode, tools_condition
from langchain.chat_models import init_chat_model

async def main():
    client = MultiServerMCPClient(
        {
            "shell": {
                "command": "python",
                "args": ["agent\\mcp_tools\\mcp_server.py"],
                "transport": "stdio",
            }
        }
    )
    tools = await client.get_tools()

    # 3️⃣ Node gọi model
    def call_model(state: MessagesState):
        response = model.bind_tools(tools).invoke(state["messages"])
        return {"messages": response}

    # 4️⃣ Xây dựng LangGraph workflow
    builder = StateGraph(MessagesState)
    builder.add_node(call_model)
    builder.add_node(ToolNode(tools))
    builder.add_edge(START, "call_model")
    builder.add_conditional_edges("call_model", tools_condition)
    builder.add_edge("tools", "call_model")
    graph = builder.compile()

    # 5️⃣ Ví dụ prompt LLM gửi shell command
    shell_prompt = "Hãy thực thi lệnh shell `echo Hello from LangGraph` và trả kết quả stdout."

    output = await graph.ainvoke({"messages": shell_prompt})
    print("Output:", output)

asyncio.run(main())


RuntimeError: asyncio.run() cannot be called from a running event loop

In [9]:
from ddgs import DDGS
import requests
from bs4 import BeautifulSoup

def search_and_fetch(query: str, region: str = "wt-wt", max_results: int = 3):
    """
    Tìm kiếm trên DuckDuckGo và tải nội dung (body text) từ các URL kết quả.
    Trả về list dict [{title, url, snippet, body}, ...]
    """
    results_data = []
    print(f"--- Đang tìm '{query}' (region={region}) ---")

    try:
        with DDGS() as ddgs:
            results = ddgs.text(query, region=region, max_results=max_results)
    except Exception as e:
        return [{"error": f"Lỗi khi gọi DuckDuckGo: {e}"}]

    if not results:
        return [{"error": "Không tìm thấy kết quả."}]

    for r in results:
        title = r.get("title") or ""
        snippet = r.get("snippet") or ""
        url = r.get("href") or None
        body_text = ""

        if url:
            try:
                resp = requests.get(url, timeout=10, headers={"User-Agent": "Mozilla/5.0"})
                resp.raise_for_status()
                soup = BeautifulSoup(resp.text, "html.parser")
                body_text = soup.get_text(separator="\n", strip=True)
            except Exception as e:
                body_text = f"[Fetch error: {e}]"

        results_data.append({
            "title": title,
            "url": url,
            "snippet": snippet,
            "body": body_text
        })

    return results_data

In [ ]:
results = search_and_fetch("luật nghĩa vụ quân sự việt nam", region="vn-vi", max_results=2)

for r in results:
    print("Title:", r["title"])
    print("URL:", r["url"])
    print("Snippet:", r["snippet"])
    print("Body preview:", r["body"][:1000], "...")  
    print("="*80)